In [1]:
from gensim.models import Word2Vec
import pandas as pd
import numpy

In [2]:
df = pd.read_csv(r'D:\PESU Assignments\Sem 4\MINI PROJECT\SSL-Offensive-Lang-Detection-Social-Media\olid-training-v1.0.tsv', delimiter='\t')
df['tweet'] = df['tweet'].str.replace('@USER','')
df

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,She should ask a few native Americans what th...,OFF,UNT,NaN
1,90194,Go home you’re drunk!!! #MAGA #Trump2020 👊🇺...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"Someone should'veTaken"" this piece of shit to...",OFF,UNT,NaN
4,43605,Obama wanted liberals &amp; illegals to move...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,Sometimes I get strong vibes from people and ...,OFF,TIN,IND
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN
13237,82921,And why report this garbage. We don't give a...,OFF,TIN,OTH
13238,27429,Pussy,OFF,UNT,NaN


In [3]:
import re
def hyperlink(tweet):
    return re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

def retweets(tweet):
    return re.sub(r'^RT[\s]+', '', tweet)

def split_hashtag(tweet):
    return re.sub(r'#', '', tweet)

def join_words(tweet):
    return re.sub(r"([a-z\.!?])([A-Z])", r"\1 \2", tweet)

df['tweet'] = df['tweet'].apply(hyperlink)
df['tweet'] = df['tweet'].apply(retweets)
df['tweet'] = df['tweet'].apply(split_hashtag)
df['tweet'] = df['tweet'].apply(join_words)
df

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,She should ask a few native Americans what th...,OFF,UNT,NaN
1,90194,Go home you’re drunk!!! MAGA Trump2020 👊🇺🇸👊...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"Someone should've Taken"" this piece of shit t...",OFF,UNT,NaN
4,43605,Obama wanted liberals &amp; illegals to move...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,Sometimes I get strong vibes from people and ...,OFF,TIN,IND
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN
13237,82921,And why report this garbage. We don't give a...,OFF,TIN,OTH
13238,27429,Pussy,OFF,UNT,NaN


In [4]:
df['tweet'] = df['tweet'].replace('\d+', '', regex=True)

In [5]:
import string

df['tweet'] = df['tweet'].str.replace('[{}]'.format(string.punctuation), '')
df['tweet'] = df['tweet'].str.strip()
df['tweet'] = df['tweet'].str.lower()

from nltk.corpus import stopwords  
from nltk.tokenize import TweetTokenizer 
tokenizer = TweetTokenizer(preserve_case=True, strip_handles=True, reduce_len=True)

df

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,she should ask a few native americans what the...,OFF,UNT,NaN
1,90194,go home you’re drunk maga trump 👊🇺🇸👊 url,OFF,TIN,IND
2,16820,amazon is investigating chinese employees who ...,NOT,NaN,NaN
3,62688,someone shouldve taken this piece of shit to a...,OFF,UNT,NaN
4,43605,obama wanted liberals amp illegals to move int...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,sometimes i get strong vibes from people and t...,OFF,TIN,IND
13236,67210,benidorm ✅ creamfields ✅ maga ✅ not too sh...,NOT,NaN,NaN
13237,82921,and why report this garbage we dont give a crap,OFF,TIN,OTH
13238,27429,pussy,OFF,UNT,NaN


In [6]:
df = df.drop(['id', 'subtask_b', 'subtask_c'], axis=1)
df = df.rename(columns={'subtask_a': 'offensive'})
df

,tweet,offensive
0,she should ask a few native americans what the...,OFF
1,go home you’re drunk maga trump 👊🇺🇸👊 url,OFF
2,amazon is investigating chinese employees who ...,NOT
3,someone shouldve taken this piece of shit to a...,OFF
4,obama wanted liberals amp illegals to move int...,NOT
...,...,...
13235,sometimes i get strong vibes from people and t...,OFF
13236,benidorm ✅ creamfields ✅ maga ✅ not too sh...,NOT
13237,and why report this garbage we dont give a crap,OFF
13238,pussy,OFF


In [7]:
def repl(off):
    if off == 'OFF':
        return 1
    return 0

df['offensive'] = df['offensive'].apply(repl)

df

,tweet,offensive
0,she should ask a few native americans what the...,1
1,go home you’re drunk maga trump 👊🇺🇸👊 url,1
2,amazon is investigating chinese employees who ...,0
3,someone shouldve taken this piece of shit to a...,1
4,obama wanted liberals amp illegals to move int...,0
...,...,...
13235,sometimes i get strong vibes from people and t...,1
13236,benidorm ✅ creamfields ✅ maga ✅ not too sh...,0
13237,and why report this garbage we dont give a crap,1
13238,pussy,1


In [8]:

tweets = [tokenizer.tokenize(tw) for tw in df['tweet']]
tweets

[['she',
  'should',
  'ask',
  'a',
  'few',
  'native',
  'americans',
  'what',
  'their',
  'take',
  'on',
  'this',
  'is'],
 ['go',
  'home',
  'you',
  '’',
  're',
  'drunk',
  'maga',
  'trump',
  '👊',
  '🇺',
  '🇸',
  '👊',
  'url'],
 ['amazon',
  'is',
  'investigating',
  'chinese',
  'employees',
  'who',
  'are',
  'selling',
  'internal',
  'data',
  'to',
  'thirdparty',
  'sellers',
  'looking',
  'for',
  'an',
  'edge',
  'in',
  'the',
  'competitive',
  'marketplace',
  'url',
  'amazon',
  'maga',
  'kag',
  'china',
  'tcot'],
 ['someone',
  'shouldve',
  'taken',
  'this',
  'piece',
  'of',
  'shit',
  'to',
  'a',
  'volcano',
  '😂'],
 ['obama',
  'wanted',
  'liberals',
  'amp',
  'illegals',
  'to',
  'move',
  'into',
  'red',
  'states'],
 ['liberals', 'are', 'all', 'kookoo'],
 ['oh', 'noes', 'tough', 'shit'],
 ['was',
  'literally',
  'just',
  'talking',
  'about',
  'this',
  'lol',
  'all',
  'mass',
  'shootings',
  'like',
  'that',
  'have',
  'been'

In [9]:
model = Word2Vec(tweets, min_count=1, size=32)

In [12]:
model.wv.most_similar('man')

[('very', 0.9952606558799744),
 ('such', 0.9948368668556213),
 ('asshole', 0.994221568107605),
 ('lying', 0.9936095476150513),
 ('ugly', 0.9921469688415527),
 ('fool', 0.9920746088027954),
 ('yes', 0.9914337396621704),
 ('shes', 0.9912853240966797),
 ('incredible', 0.9911694526672363),
 ('fan', 0.9910383224487305)]

In [18]:
vec = model['king'] - model['man'] + model['woman']
model.wv.most_similar([vec])

C:\ProgramData\Anaconda3\envs\opencv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


[('strongest', 0.8275538086891174),
 ('manufacturers', 0.8239356279373169),
 ('contravention', 0.801516056060791),
 ('liberties', 0.7931129932403564),
 ('pitch', 0.7743411660194397),
 ('provision', 0.7736194729804993),
 ('freakazoids', 0.7698692083358765),
 ('frontline', 0.7663330435752869),
 ('restrictive', 0.7542758584022522),
 ('sudden', 0.7222731113433838)]

In [ ]:
model1 = Word2Vec(tweets, )